In [1]:
# Importing the nodes and edges, Graph generation
import networkx as nx
import pandas as pd
G = nx.Graph()
nodes = pd.read_csv('nodes.csv')
nodes.head(5)
edges = pd.read_csv('5-edges.csv')
edges.head(5)
G.add_nodes_from(nodes)
for i, elrow in edges.iterrows():
    G.add_edge(elrow[0], elrow[1], attr_dict=elrow[2:].to_dict())
    
print('Number of edges: {}'.format(G.number_of_edges()))
print('Number of nodes: {}'.format(G.number_of_nodes()))

######################################################################

# Graph Generation
#import matplotlib.pyplot as plt
#plt.figure(figsize=(8, 6))
#nx.draw(G)

#####################################################################
# Reducing the network to smaller one.
__all__ = ['degree_centrality']
def degree_centrality(G):

    if len(G) <= 1:
        return {n: 1 for n in G}

    s = 1.0 / (len(G) - 1.0)
    centrality = {n: d * s for n, d in G.degree()}
    return centrality
dc = degree_centrality(G)

attack = {k:v for (k,v) in dc.items() if v>0.01}
G.remove_nodes_from(attack)
print('Reduced Number of edges: {}'.format(G.number_of_edges()))
print('Reduced Number of nodes: {}'.format(G.number_of_nodes()))

######################################################################

# Graph Generation
#import matplotlib.pyplot as plt
#plt.figure(figsize=(8, 6))
#nx.draw(G)

##############################################################
def degree_centrality(G):

    if len(G) <= 1:
        return {n: 1 for n in G}

    s = 1.0 / (len(G) - 1.0)
    centrality = {n: d * s for n, d in G.degree()}
    return centrality
dc = degree_centrality(G)

##############################################################
__all__ = ['closeness_centrality']


def closeness_centrality(G, u=None, distance=None,
                         wf_improved=True, reverse=False):

    if distance is not None:
        # use Dijkstra's algorithm with specified attribute as edge weight
        path_length = functools.partial(nx.single_source_dijkstra_path_length,
                                        weight=distance)
    else:  # handle either directed or undirected
        if G.is_directed() and not reverse:
            path_length = nx.single_target_shortest_path_length
        else:
            path_length = nx.single_source_shortest_path_length

    if u is None:
        nodes = G.nodes()
    else:
        nodes = [u]
    closeness_centrality = {}
    for n in nodes:
        sp = dict(path_length(G, n))
        totsp = sum(sp.values())
        if totsp > 0.0 and len(G) > 1:
            closeness_centrality[n] = (len(sp) - 1.0) / totsp
            # normalize to number of nodes-1 in connected part
            if wf_improved:
                s = (len(sp) - 1.0) / (len(G) - 1)
                closeness_centrality[n] *= s
        else:
            closeness_centrality[n] = 0.0
    if u is not None:
        return closeness_centrality[u]
    else:
        return closeness_centrality


closeness = closeness_centrality(G)

######################################################################
attack = {k:v for (k,v) in dc.items() if v>0.007}
defend = {k:v for (k,v) in closeness.items() if v>0.01}

attack_list = list(attack.keys())
defend_list = list(defend.keys())

def Diff(attack_list, defend_list):
    li_diff = [i for i in attack_list + defend_list if i not in attack_list or i not in defend_list]
    return li_diff

li3 = Diff(attack_list, defend_list)

#print(li3)

G.remove_nodes_from(li3)

######################################################################
# Calculation of Number Of Maximal Cliques
from collections import deque
from itertools import chain
from itertools import combinations
from itertools import islice
try:
    from itertools import ifilter as filter
except ImportError:
    pass
import networkx as nx
from networkx.utils import not_implemented_for
__all__ = ['find_cliques',
           'maximal_cliques']

def find_cliques(G):
    
    if len(G) == 0:
        return

    adjacent = {u: {v for v in G[u] if v != u} for u in G}
    Q = [None]

    subg = set(G)
    cand = set(G)
    u = max(subg, key=lambda u: len(cand & adjacent[u]))
    ext_u = cand - adjacent[u]
    stack = []

    try:
        while True:
            if ext_u:
                q = ext_u.pop()
                cand.remove(q)
                Q[-1] = q
                adjacent_q = adjacent[q]
                subg_q = subg & adjacent_q
                if not subg_q:
                    yield Q[:]
                else:
                    cand_q = cand & adjacent_q
                    if cand_q:
                        stack.append((subg, cand, ext_u))
                        Q.append(None)
                        subg = subg_q
                        cand = cand_q
                        u = max(subg, key=lambda u: len(cand & adjacent[u]))
                        ext_u = cand - adjacent[u]
            else:
                Q.pop()
                subg, cand, ext_u = stack.pop()
    except IndexError:
        pass

def maximal_cliques(G, cliques=None):
    
    if cliques is None:
        cliques = list(find_cliques(G))
    return len(cliques)


###############################################################################
ac = nx.average_clustering(G)
m_cliques = maximal_cliques(G)
print("The Defended Network's average clustering is :'",ac)
print("The Number of maximal cliques in the Defended Network is:",m_cliques)

######################################################################

# Graph Generation
#import matplotlib.pyplot as plt
#plt.figure(figsize=(8, 6))
#nx.draw(G)


Number of edges: 3797634
Number of nodes: 13161
Reduced Number of edges: 11716
Reduced Number of nodes: 5847
The Defended Network's average clustering is :' 0.03465661424187933
The Number of maximal cliques in the Defended Network is: 2169
